### import 및 전역변수

In [2]:
import os
import shutil
import time
import pandas as pd
from tqdm.notebook import tqdm      #tqdm(filelist, total = len(file_list), position=0, leave=True)
import re
from pathlib import Path
import traceback
from os.path import join
import file_function as ff
from datetime import datetime

In [ ]:
#[시작]파일업로드###########################################################
# 참조 df 불러오기 ~~~~ ":" xlsx파일에서 미리 제거해둬야 ~~~~
df_c = pd.read_excel(r'C:\Users\SL\Desktop\채무자조회.xlsx')
df_c.tail()

# 채무상태와 매각사구분만 뺀 dict
dict_sell ={}
for ind, col in df_c.iterrows():
    dict_sell[str(col.채무자키)] = [col.매각사구분, col.채무상태]

In [ ]:
# 경로 정리 ?????? 필요한가???
path = r'\\192.168.0.75\스캔파일\새 스캔파일(업로드)\파일명 수정 필요'
os.chdir(path)
file_list = [f.name for f in os.scandir(path) if f.is_file()]
_total = len(file_list)
log_path = r'\\192.168.0.75\스캔파일\스캔파일 log'

f_d = r'\\192.168.0.75\솔림헬프'
out = r'\\192.168.0.75\삭제예정파일\관리제외'

# 기본 변수
p_out = re.compile('개인회생\(면책\)|파산\(면책\)|환매|매각|종결')
p_key = re.compile(r"([\d]{8})[\D]") #match

error = []
log = []

In [2]:
#[디렉토리 만들기 시작]###########################################################
#디렉토리 만들기 - 완료
# path = r'\\192.168.0.75\솔림헬프'
# os.chdir(path)
# docu_list = ["1.원인서류", "2.양도통지서", "3.집행권원", "4.강제집행", "5.등초본", "6.개인회생", "7.신용회복", "8.파산", "9.재산조사", "기타"]
# sell_list = ["전북-01", "DNP-01", "DNP-02", "메리츠캐_01", "KNS", "DNP-03", "한울가람", "케이에스", \
#     "유안타저-01", "KB카드-01", "베리타스-01", "BNK-01", "스마트저축-01", "우담-01", "고려저축-01", \
#         "롯데카드-01", "헤럴드", "KCR", "아인스", "하나캐-02", "베델-01", "롯데카드-03", "하나캐-01", \
#             "더키움", "롯데카드-02", "JT캐-02", "흥국화재-01", "롯데카드-04", "산은캐-01", "대산", "JT캐-01"]

# for dir_ in docu_list :
#     for sell_ in sell_list :
#         os.makedirs(dir_+"/"+sell_, exist_ok=True)

#[끝]###########################################################

### 변종 통일하기

In [ ]:
# 문서구분 수정, 보증인 위치, 차수, 재도 확인, 이상한 기호 제거, 날짜 양식 확인
path = r'C:\Users\jbmyo\Desktop\test'
file_list = [f.name for f in os.scandir(path) if f.is_file()]



### 수작업 분류 및 format 재작성

In [ ]:
path = r'C:\Users\jbmyo\Desktop\test'
hard_path = r'\\192.168.0.75\스캔파일\새 스캔파일(업로드)\수작업 필요'
checked_path = r'\\192.168.0.75\스캔파일\새 스캔파일(업로드)\파일명 검수 완료'

file_list = [f.name for f in os.scandir(path) if f.is_file()]
log = []
error = []
cnt = 0

docu_kind = r'원인서류|양도통지서[\s]?[\d]?[차]?|집행권원[\s]?[재]?[도]?|집행권원 재도|집행권원재도|강제집행|등본|초본|외국인증명|개인회생|신용회복|파산|재산조사|기타'

p_basic = re.compile(r'([\d]{8})[\s]([\D]+)('+docu_kind+r')') #[\D]는 공백문자를 포함하므로 아래서 strip하는 게 낫다.
p_event = re.compile(r"[\D]([\d]{4}[ㄱ-ㅎ가-힣]{1,3}[0-9]+)")


In [ ]:
for f in file_list :
    try :
        stem = os.path.splitext(f)[0]
        ext = os.path.splitext(f)[1]

        # 필수 양식 확인
        basic = p_basic.match(stem)
        if basic == None : # 필수 양식에 맞지 않는다.
            shutil.move(join(path, f), join(hard_path, f))
            cnt += 1
        
        else :
            
            # 필수3요소
            key = basic.group(1)
            name = basic.group(2).strip() #[/D]가 공백을 포함하므로
            docu = basic.group(3)
            
            # 사건번호, 기타정보
            if p_event.search(stem) :
                temp_event = p_event.search(stem)
                event = temp_event.group()
                extra = stem[temp_event.end():].strip().replace(" ", "_")
            else :
                extra = stem[basic.end():].strip().replace(" ", "_")

            # 기타정보(없는 경우 끝에 _ 생겨서 구분 해줘야 함)
            if extra :
                name_items = [key, name, docu, event, extra]
            else :
                name_items = [key, name, docu, event]

            new_name = "_".join(name_items)+ext
            new_name = re.sub("[_]{2,}", "_", new_name) #마지막에 _ 두개 인 경우 꼭 해줘야 해.
            
            shutil.move(join(path, f), join(checked_path,new_name))
            log.append([f, new_name])
            
    except Exception as e:
        print("===================================")
        print(datetime.today().strftime("%H:%M:%S"))
        print(f, e.__class__, e, sep=" : ")
        print(traceback.format_exc())
        error.append([f, e.__class__, e])
        continue # 반복문 계속 돌아

ff.write_log(log, error, r'\\192.168.0.75\스캔파일\스캔파일 log\파일명 체크')
print(cnt, "개 파일 수작업 필요")


### "솔림헬프"로 파일 업로드

In [19]:
# 경로 정리
path = r'\\192.168.0.75\스캔파일\새 스캔파일(업로드)\파일명 수정 필요'
os.chdir(path)
file_list = [f.name for f in os.scandir(path) if f.is_file()]
_total = len(file_list)
log_path = r'\\192.168.0.75\스캔파일\스캔파일 log'

f_d = r'\\192.168.0.75\솔림헬프'
out = r'\\192.168.0.75\삭제예정파일\관리제외'
# 기본 변수
p_out = re.compile('개인회생\(면책\)|파산\(면책\)|환매|매각|종결')
p_key = re.compile(r"([\d]{8})[\D]") #match

error = []
log = []
# 1) depth1 문서종류 매칭
docu_match = {"원인서류" : "1.원인서류", "양도통지서":"2.양도통지서", "집행권원":"3.집행권원", "강제집행":"4.강제집행", "등본":"5.등초본", "초본":"5.등초본", "외국인":"5.등초본", "개인회생":"6.개인회생", "신용회복":"7.신용회복", "파산":"8.파산", "재산조사":"9.재산조사", "기타":"기타"}

In [20]:
#######################################
#로그, 관리제외파일 수정했으니 테스트 한 번 해봐
#######################################

for f in tqdm(file_list, total=_total) : 
    
    if (f != "Thumbs.db") :
        
        try :
            depth1 = "" #문서종류
            depth2 = dict_sell[depth3][0] #매각사구분
            depth3 = f[:8] #다 확인한 뒤니까 그냥 이렇게 해도 돼

            # 관리제외라면
            if p_out.match(dict_sell[depth3][1]) :
                out_dir = join(out, depth2, depth3)
                if not os.path.exists(out_dir) :
                    os.makedirs(out_dir)
                shutil.move(join(path,f), join(out_dir, f))
            
            # 관리파일이면
            else :
            
                # depth1
                for key, value in docu_match.items() : 
                    if re.search(key, f) :
                        depth1 = value
                        continue
                    else :
                        pass
                
                # 파일이동을 위한 준비(도착지 디렉토리 및 파일명 작성)
                dst_dir = os.path.join(f_d, depth1, depth2, depth3)
            
                if not os.path.exists(dst_dir) :
                    os.makedirs(dst_dir) #미리 만들어뒀으니 mkdir해도 됨
                
                #파일이동
                log.append(ff.re_name(join(path, f), join(dst_dir, f)))
        
        except Exception as e:
            print("===================================")
            print(datetime.today().strftime("%H:%M:%S"))
            print(f, e.__class__, e, sep=" : ")
            print(traceback.format_exc())
            error.append([f, e.__class__, e])
            continue # 반복문 계속 돌아


# 로그 파일 나스에 저장
ff.write_log(log, error, log_path)

  0%|          | 0/118 [00:00<?, ?it/s]

88 개의 파일 업로드 완료, 0 개의 요확인 파일 발생


### 파일 비교 및 차집합

In [ ]:
myPath= r'D:\부산 정리_v2'
nasPath=r'\\192.168.0.75\스캔파일\새 스캔파일(업로드)'

In [ ]:
myFile = [f for f in os.scandir(myPath) if f.is_file()]

In [ ]:
nasFile = [f for f in os.scandir(nasPath) if f.is_file()]

In [ ]:
onlyLocal = list(set(myFile) - set(nasFile))
print(len(onlyLocal))

In [ ]:
onlynas = list(set(myFile) - set(nasFile))
print(len(onlynas))

hello
